# 第8章: ニューラルネット

第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

In [ ]:
#colabを利用する上での前準備
from google.colab import drive
drive.mount('/content/drive')

In [2]:
cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [3]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 70. 単語ベクトルの和による特徴量

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例xi
の特徴ベクトルxi
を並べた行列X
と，正解ラベルを並べた行列（ベクトル）Y
を作成したい．

ここで，n
は学習データの事例数であり，xi∈ℝd
とyi∈ℕ
はそれぞれ，i∈{1,…,n}
番目の事例の特徴量ベクトルと正解ラベルを表す． なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．ℕ<4
で4
未満の自然数（0
を含む）を表すことにすれば，任意の事例の正解ラベルyi
はyi∈ℕ<4
で表現できる． 以降では，ラベルの種類数をL
で表す（今回の分類タスクではL=4
である）．

i
番目の事例の特徴ベクトルxi
は，次式で求める．

ここで，i
番目の事例はTi
個の（記事見出しの）単語列(wi,1,wi,2,…,wi,Ti)
から構成され，emb(w)∈ℝd
は単語w
に対応する単語ベクトル（次元数はd
）である．すなわち，i
番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものがxi
である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．300
次元の単語ベクトルを用いたので，d=300
である．

i
番目の事例のラベルyi
は，次のように定義する．

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

学習データの特徴量行列: Xtrain∈ℝNt×d
学習データのラベルベクトル: Ytrain∈ℕNt
検証データの特徴量行列: Xvalid∈ℝNv×d
検証データのラベルベクトル: Yvalid∈ℕNv
評価データの特徴量行列: Xtest∈ℝNe×d
評価データのラベルベクトル: Ytest∈ℕNe
なお，Nt,Nv,Ne
はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．



In [20]:
#50と同様

import pandas as pd
from sklearn.model_selection import train_test_split

#データ読み込み #エラーが出るため、タブ区切り、ラベル名を指定する
df = pd.read_csv('./NewsAggregatorDataset/newsCorpora.csv', sep="\t", names=["id", "title", "url", "publisher", "category", "story", "hostname", "timestamp"])

#情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する
df = df[df['publisher'].isin(["Reuters", "Huffington Post", "Businessweek", "COntactmusic.com", "Daily Mail"])]

#抽出された事例をランダムに並び替える
df = df.sample(frac = 1, random_state = 42, ignore_index=True)

#抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する
train, test = train_test_split(df, test_size=0.2, random_state = 42)
test, valid = train_test_split(test, test_size=0.5, random_state = 42)

In [7]:
#60と同様

import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('./shogi/GoogleNews-vectors-negative300.bin.gz', binary= True)

In [8]:
import torch

train['title']

In [32]:
lis_train_vecs =[] #1文内の単語ベクトルのリスト
tensor_vecs = [] #平均単語ベクトルのリスト
for title in train['title']:
  lis_train_words = title.split()
  for word in lis_train_words:
    try:
      vec = model[word]
      lis_train_vecs.append(vec)

    except: #未知語の場合
      pass

  tensor_vec = torch.tensor(sum(lis_train_vecs) / len(lis_train_vecs))
  tensor_vecs.append(tensor_vec)
  lis_train_vecs =[]


In [39]:
lis_train_vecs[1]

array([ 7.91015625e-02,  9.66796875e-02, -1.08642578e-02,  2.56347656e-02,
       -3.24707031e-02,  1.09863281e-01, -1.12792969e-01, -4.44335938e-02,
        1.07421875e-02, -8.30078125e-03,  2.07519531e-02, -2.78320312e-02,
       -7.95898438e-02, -4.93164062e-02, -1.49414062e-01,  5.71289062e-02,
        1.57226562e-01, -1.85546875e-02, -1.24023438e-01, -3.36914062e-02,
       -3.50952148e-03,  3.68652344e-02, -2.03857422e-02, -1.08398438e-01,
        7.47070312e-02, -8.74023438e-02, -1.53320312e-01,  1.07910156e-01,
       -2.53906250e-02, -1.08886719e-01,  7.51953125e-02,  1.17675781e-01,
        4.88281250e-02,  3.33786011e-04,  1.22558594e-01, -5.37109375e-02,
        9.27734375e-02, -3.39355469e-02, -1.06445312e-01,  9.47265625e-02,
        4.05273438e-02,  6.25000000e-02,  4.05273438e-02, -6.49414062e-02,
       -3.83300781e-02, -3.14941406e-02, -4.73632812e-02, -1.86767578e-02,
        1.42211914e-02,  2.90527344e-02, -8.88671875e-02,  1.64062500e-01,
        3.56445312e-02, -

In [ ]:
# ターゲットのテンソル化
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
Y_train = torch.from_numpy(train['category'].map(category_dict).values)
Y_valid = torch.from_numpy(valid['category'].map(category_dict).values)
Y_test = torch.from_numpy(test['category'].map(category_dict).values)
# 保存
torch.save(X_train, './NewsAggregatorDataset/X_train.pt')
torch.save(X_valid, './NewsAggregatorDataset/X_valid.pt')
torch.save(X_test, './NewsAggregatorDataset/X_test.pt')
torch.save(Y_train, './NewsAggregatorDataset/y_train.pt')
torch.save(Y_valid, './NewsAggregatorDataset/y_valid.pt')
torch.save(Y_test, './NewsAggregatorDataset/y_test.pt')

In [34]:
torch.stack(tensor_vecs)

tensor([ 0.0115,  0.0006,  0.0168,  ..., -0.0010,  0.0009,  0.0061],
       dtype=torch.float64)

# 72. 損失と勾配の計算

学習データの事例x1
と事例集合x1,x2,x3,x4
に対して，クロスエントロピー損失と，行列W
に対する勾配を計算せよ．なお，ある事例xi
に対して損失は次式で計算される．

li=−log[事例xiがyiに分類される確率]
ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．



# 73. 確率的勾配降下法による学習